In [35]:
import os
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth
client_id = SPOTIPY_CLIENT_ID ='bb7c475418484e7784d9cd25b5f9f52c'
client_secret = SPOTIPY_CLIENT_SECRET='b0da0baeeab1499884912aea11f4ca58'
redirect_uri =SPOTIPY_REDIRECT_URI='https://localhost:8080/callback/'


os.environ["SPOTIPY_CLIENT_ID"] = "bb7c475418484e7784d9cd25b5f9f52c"
os.environ["SPOTIPY_CLIENT_SECRET"] = "b0da0baeeab1499884912aea11f4ca58"
os.environ["SPOTIPY_REDIRECT_URI"] = "https://localhost:8080/callback/"

scope = "playlist-modify-public playlist-modify-private user-modify-playback-state user-top-read"
scope +=            " user-modify-playback-state user-read-playback-state user-library-read user-library-modify"
username = 'tonyryanworldwide'

        
token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)        
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope,username=username))


Enter the URL you were redirected to: https://localhost:8080/callback/?code=AQAnGxzwRZk2W7yjCfsNwpljIBNK9JwgZlTxWdntv8zHp6z2_nG9Eq7HnSUgBL25goGihD21w1VaZcbDtoTCWGQGgMbskWw7OKlJXgvjtrU2RrEpl-bVnl5NiQlpgr_YtMQyMyWYRuvayQueB-1r0UX80OwvdVvty_jMASXgFDn-dx_qKmeKKaVE5Dzb_GtCCJnUqAhdr83VxuWtraK0CTLB9tRN9h7h7cmjKUdHBcXj_tqMF0gndXY8jgomljnrnm60Le3LUWTzBNTKRKSegOuCCGfEAWsWo-KuBM4SgQtmAijX4JWLgyWOZS1yh0EghGXPbylj2yjVu7WuPgHzrWBiq7QsZiNnlFTxGev4_vldptbJZovn8sAzX8U2eRbGJv4


In [12]:
import pandas as pd
from pandas import DataFrame
total = sp.current_user_saved_tracks()['total']
offset = 50
savedtracks = sp.current_user_saved_tracks(limit = 50)['items']
while offset < total:
    
    savedtracks2 =sp.current_user_saved_tracks(limit = 50,offset=offset)['items']
    for track in savedtracks2:
        savedtracks.append(track)
    offset += 50

    
trackinfo ={}   
for trackex in savedtracks:
    id = trackex['track']['id']
    name = trackex['track']['name']
    artistid = trackex['track']['artists'][0]['id']
    seconds = trackex['track']['duration_ms']/1000
    trackinfo[trackex['track']['id']] = {"artist" : artistid,"name":name,"seconds":seconds}
    
trackdf = pd.DataFrame.from_dict(trackinfo,orient = "index").reset_index()
artists = trackdf.artist.unique()
trackdf.columns = ['trackid','artistid','trackname','tracklength_sec']

artistinfo = []
i = 0
while i < len(artists) - 50:
    x=sp.artists(artists[i:i+50])['artists']
    artistinfo.append(x)
    i+=50
artistfinal ={}
for i in artistinfo:
    for j in i:   
        id = j['id']
        name = j['name']
        if len(j['genres']) > 0:
            genre = j['genres']#[0]
        else:
            genre = ''
        artistfinal[id] = {"name":name,"genre": genre}          
artistdf = pd.DataFrame.from_dict(artistfinal,orient = "index").reset_index()
artistdf = artistdf.reset_index()[['index','name','genre']]
artistdf.columns = ['artistid','artistname','genre']
artistdf =artistdf[artistdf['genre']!=''].reset_index()[['artistid','artistname','genre']]

Enter the URL you were redirected to: https://localhost:8080/callback/?code=AQDt0pEGJzJm_pgePNV4B0nk65oOR5oC-fF2uQSiyKwpfaOdV0XVku6PjZHIbeRmvt4hA9JDtWKnwxavoTEZcYf5EBSHIeK2IINMQYYdPJMhwPdWJYSV9-crKhw52Avz6TySAZjWQM4OabcjTpFdjOV6qr_CtbRL_omb_9Q3BgwvyAEhklI0vb-ODz7lbXMCqaUzG4BwOAMiVBm1PAjLMATr7TkW7KRpGLakqbue1ZEWQMoO2vwUDWV30A2HkOvud9LppAI7BKuGY6dAaHHM6lgVm2Kf8wRm70Yq1xqgDSOdE3jZ3ghs6A8CCgI96YN0iAWMiYZ3ZNjCwPTZlxCtKti6zc7xX1oM3xCISpTvnSUKTlCwgrPRMmAkRzeNu6jZ16U


In [13]:
column_names = ["artistid", "artistname", "genre"]

finalartist = pd.DataFrame(columns = column_names)

for i in range(0, len(artistdf)):
    df = artistdf[i:i+1][['artistid','artistname']]
#     print(i)
#     print(df)
    genres = artistdf[i:i+1]['genre'][i]
    gf= DataFrame(genres,columns=['genre'])
    product = (
    df.assign(key=1)
    .merge(gf.assign(key=1), on="key")
    .drop("key", axis=1)
    )
    finalartist = finalartist.append(product)
    

In [14]:
finaltrackinfo = trackdf.merge(finalartist, left_on = 'artistid', right_on = 'artistid', how = 'inner' )
finaltrackinfo['cum_sec'] = finaltrackinfo.groupby('genre')['tracklength_sec'].apply(lambda x: x.cumsum())

In [15]:
genres = finaltrackinfo.groupby('genre').count().reset_index()[['genre','artistid']]
genres.columns=['genre','count']
genres = genres[(genres['count'] >= 25) & (genres['genre']!= '')].sort_values(by ='count', ascending = False)
genrelist = genres['genre'].tolist()
finaltrackinfo = finaltrackinfo[finaltrackinfo['genre'].isin(genres['genre'])]

In [16]:
def genrefilter(df,gen,sec_lim = 36000):
    dfin = df['trackid'][(df['genre'] == gen) & (df['cum_sec'] <= sec_lim)]
    return dfin.tolist()#[0:100]



In [17]:
# for gen in genrelist:
#     print(gen)
#     tracks = genrefilter(finaltrackinfo,gen,36000)
#     print(len(tracks))

gen = genrelist[9]
print(gen)
tracks = genrefilter(finaltrackinfo,gen)
id = sp.user_playlist_create(user=username,name =gen)['id']
len(tracks)

hip hop


136

In [18]:
for gen in genrelist:
    id = sp.user_playlist_create(user=username,name =gen)['id']
    tracks = genrefilter(finaltrackinfo,gen)
#     sp.user_playlist_add_tracks(user = username, playlist_id =id,tracks = tracks,position = 0)
    lengthtracks = len(tracks)
    x= 0
    if lengthtracks <= 100:
        y = lengthtracks
        print("x:{0} y:{1}".format(x,y))
        sp.user_playlist_add_tracks(user = username, playlist_id =id,tracks = tracks[x:y],position = 0)
    else:
        y = 100
        print("1st else: x{} y{}".format(x,y))
        sp.user_playlist_add_tracks(user = username, playlist_id =id,tracks = tracks[x:y],position = 0)
        iteration = 1
        while y >0:
            x = x +100
            if lengthtracks < x  + 100:
                y = lengthtracks   
                print("inif: x{} y{}".format(x,y))
                sp.user_playlist_add_tracks(user = username, playlist_id =id,tracks = tracks[x:y],position = 0)
                y = 0
            else:
                y = y + 100
                print("else x:{} y:{}".format(x,y))
                sp.user_playlist_add_tracks(user = username, playlist_id =id,tracks = tracks[x:y],position = 0)
            iteration += 1

1st else: x0 y100
inif: x100 y138
1st else: x0 y100
inif: x100 y145
1st else: x0 y100
inif: x100 y155
1st else: x0 y100
inif: x100 y126
1st else: x0 y100
inif: x100 y123
1st else: x0 y100
inif: x100 y141
1st else: x0 y100
inif: x100 y141
1st else: x0 y100
inif: x100 y143
1st else: x0 y100
inif: x100 y124
1st else: x0 y100
inif: x100 y136
1st else: x0 y100
inif: x100 y125
1st else: x0 y100
inif: x100 y110
1st else: x0 y100
inif: x100 y128
1st else: x0 y100
inif: x100 y119
1st else: x0 y100
inif: x100 y114
1st else: x0 y100
inif: x100 y109
1st else: x0 y100
inif: x100 y106
1st else: x0 y100
inif: x100 y102
1st else: x0 y100
inif: x100 y101
x:0 y:93
x:0 y:91
x:0 y:90
x:0 y:89
x:0 y:87
x:0 y:83
x:0 y:80
x:0 y:77
x:0 y:75
x:0 y:74
x:0 y:73
x:0 y:72
x:0 y:72
x:0 y:66
x:0 y:61
x:0 y:59
x:0 y:58
x:0 y:58
x:0 y:57
x:0 y:56
x:0 y:56
x:0 y:55
x:0 y:53
x:0 y:52
x:0 y:51
x:0 y:50
x:0 y:50
x:0 y:48
x:0 y:43
x:0 y:43
x:0 y:43
x:0 y:42
x:0 y:42
x:0 y:42
x:0 y:38
x:0 y:36
x:0 y:35
x:0 y:35
x:0 y:33
x:0

## For Debugging Delete CreatedPlaylists

In [30]:
# playlists =sp.current_user_playlists(offset=100)['items']
playlists = sp.user_playlists(username,offset=50)['items']
for playlist in playlists:
    name= playlist['name']
    print(name)
    
    id = playlist['uri'][-22:]
    
    if name in genrelist:
        print('deleting{0} id {1}'.format(name,id))
        sp.user_playlist_unfollow(user=username, playlist_id = id)

alternative pop
deletingalternative pop id 3Waw1ls9CES2WpYMajiI2F
folk rock
deletingfolk rock id 1XCbwzjjM5GcTuTNVgX7uV
power pop
deletingpower pop id 2mdJZUQK6iCHPLqo2sCGd4
roots rock
deletingroots rock id 1FZIkfuhuLzeQcqJcB9Lx1
soft rock
deletingsoft rock id 7ibJ9DxzA2iTmzvvNVUK8w
nu metal
deletingnu metal id 4hJO2obgH2xdbvyBxHoe9R
neo mellow
deletingneo mellow id 6wvkkKoqo5rmu9b5LpeNnV
stomp and holler
deletingstomp and holler id 1ZRZIeZGevkS1hZecrGHuF
post-grunge
deletingpost-grunge id 1hZqyX29wl2AySlVdWRDiZ
mellow gold
deletingmellow gold id 6JY1HVKqz9d9LZfkrzy5Y0
art rock
deletingart rock id 0CF9q1nOGEqVgf6JnIly0C
alternative metal
deletingalternative metal id 6oa0LwrASUW0x8IjMHHc0w
hip hop
deletinghip hop id 4UyUAFd3echux26NCKurf9
hard rock
deletinghard rock id 5DXueOKY1WKBrAfvRFY3Sh
permanent wave
deletingpermanent wave id 3DqYG0RuPoBc8TWlC8jG4j
rap
deletingrap id 3tU17S0QNuwTLuxbdOAq2S
pop rock
deletingpop rock id 0YEGJ6jhUTY7Gv4umrpGUZ
album rock
deletingalbum rock id 5ZwLTqx

In [33]:
offset =0
playlistlength =50
while(playlistlength == 50):   
    playlists = sp.user_playlists(username,offset=offset)['items']
    playlistlength = len(playlists)
    print("playlistlength {0}".format(playlistlength))
    for playlist in playlists:
        name= playlist['name']
        print(name)
        id = playlist['uri'][-22:]
    
        if name in genrelist:
            print('deleting{0} id {1}'.format(name,id))
            sp.user_playlist_unfollow(user=username, playlist_id = id)
    offset += 50
    

playlistlength 50
post-punk
deletingpost-punk id 4ZlvjTIFAkifCQDepc6RkD
country road
deletingcountry road id 2SDQURRiv7GIGFy7mANGQO
rap rock
deletingrap rock id 2WBXaK8yOC7s8TZZGFInxA
country
deletingcountry id 1g5xPkOay9YM4Suo3f9udk
contemporary country
deletingcontemporary country id 6iwetuuwIDUySRM6zp0pN8
indie folk
deletingindie folk id 0jj2q8WxqmEewmRfg1cK83
canadian rock
deletingcanadian rock id 4kNNeLk3pas0FGHFtKKJv8
chicago rap
deletingchicago rap id 54ufWc1BrzjnYF8te2KWvh
supergroup
deletingsupergroup id 2M1VoQw2Lb6BYmsYGIomAu
queens hip hop
deletingqueens hip hop id 5cBOn45ulqTGxDdiJ2Ip6v
skate punk
deletingskate punk id 6NTdXAoGBdlN3IdUBZcoWA
north carolina hip hop
deletingnorth carolina hip hop id 6CMLQkgHv310xX13dk5Xsl
modern alternative rock
deletingmodern alternative rock id 6sbnF86A4PkRbCBkvV3SSf
irish rock
deletingirish rock id 4x4Fr9oBdl4sffylYuAEZj
symphonic rock
deletingsymphonic rock id 3C9o89a9GIW3gRHXrpbkgZ
new wave
deletingnew wave id 5LHpXdeSrJxlh8EsiBau0x
sout

In [31]:
playlists = sp.user_playlists(username,offset=50)['items']
for playlist in playlists:
    name= playlist['name']
    print(name)

sexy time
indie folk
heartland rock
glam metal
gbvfi
folk-pop
folk rock
dance rock
canadian rock
art rock
alternative rock
alternative metal
Generator
Small Town and bruce offshoot
A Star Is Born
Small Town and bruce
Good classical
j cole
upbeat
Emma
Most People Are Good
dad stuff
Weird female vocals
Sophie List
K.Dot
Cadence To Arms
work
Run
2pac


In [ ]:
playlists = sp.user_playlists(username,offset=offset)['items']